In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
from elasticsearch import Elasticsearch

In [8]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
es_client = Elasticsearch('http://localhost:9200')

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [9]:
for doc in tqdm(documents):
    es_client.index(index = index_name, document = doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:21<00:00, 44.42it/s]


In [10]:
query = "How do I execute a command in a running docker container?"

In [17]:
def elastic_search(query):

    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index = index_name, body= search_query)
    result_docs  = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])

    return result_docs

In [18]:
elastic_search(query)

[84.050095, 75.54128, 72.08518, 51.04628, 49.938507]

In [21]:
def elastic_search(query):

    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index = index_name, body= search_query)
    result_docs  = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [23]:
results = elastic_search(query)

In [25]:
results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [26]:
context_template = """
Q: {question}
A: {text}
""".strip()

# Assuming 'results' is the list of records from Elasticsearch
formatted_contexts = []

for record in results:
    formatted_context = context_template.format(
        question=record['question'],
        text=record['text']
    )
    formatted_contexts.append(formatted_context)

# Join all formatted contexts with two newline characters
final_context = "\n\n".join(formatted_contexts)




In [27]:
print(final_context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [28]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [29]:
prompt = prompt_template.format(question=query, context=final_context).strip()

In [31]:
len(prompt)

1462

In [35]:
import tiktoken

In [36]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [38]:
len(encoding.encode(prompt))

322

In [39]:
import os
from openai import OpenAI

In [40]:
os.environ["OPENAI_API_KEY"] = "sk-"

In [41]:
api_key = os.environ.get('OPENAI_API_KEY')

In [44]:
client = OpenAI(api_key=api_key)

In [45]:
def llm(prompt):
    response = client.chat.completions.create(model="gpt-4o", 
                                          messages = [{"role":"user", "content":prompt}])
    
    return response.choices[0].message.content

In [49]:
response = (llm(prompt))

In [53]:
print(response)

To execute a command in a running Docker container, you can use the `docker exec` command. Here are the steps:

1. List all running Docker containers to find the container ID for the specific container you want to execute a command in:
   ```
   docker ps
   ```

2. Execute a command inside the container using the container ID:
   ```
   docker exec -it <container-id> bash
   ```

Replace `<container-id>` with the actual ID of your running container. This command will open a bash shell in the specified container, where you can then execute your desired commands.


In [50]:
len(response)

568

In [51]:
len(encoding.encode(response))

122

To calculate the cost of running 1000 requests with the given information, let's break it down step by step:

1. Calculate the total number of tokens:
   - Input tokens per request: 322
   - Output tokens per request: 122
   - Total tokens per request: 322 + 122 = 444
   - For 1000 requests: 444 * 1000 = 444,000 tokens

2. Separate input and output tokens:
   - Total input tokens: 322 * 1000 = 322,000
   - Total output tokens: 122 * 1000 = 122,000

3. Calculate the cost for input tokens:
   - Price: $0.005 per 1K tokens
   - Cost: (322,000 / 1000) * $0.005 = $1.61

4. Calculate the cost for output tokens:
   - Price: $0.015 per 1K tokens
   - Cost: (122,000 / 1000) * $0.015 = $1.83

5. Sum up the total cost:
   - Total cost: $1.61 + $1.83 = $3.44

Therefore, the cost to run 1000 requests with the given token counts and prices would be $3.44.